<a href="https://colab.research.google.com/github/Vaish181204/Suvidha-Foundation/blob/main/Gemma_2_9B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes accelerate transformers peft evaluate rouge_score bert-score

In [2]:
# 1. Preferred installation method to avoid colab-new errors
!pip install unsloth
# 2. Force install dependencies to ensure compatibility
!pip install --no-deps xformers trl peft accelerate bitsandbytes
# 3. Required for metrics
!pip install -q -U evaluate rouge_score bert-score nltk

  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.36.2-py3-none-any.whl.metadata (15 kB)
Using cached transformers-4.57.6-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.36.2-py3-none-any.whl (566 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.4.1
    Uninstalling huggingface_hub-1.4.1:
      Successfully uninstalled huggingface_hub-1.4.1
  Attempting uninstall: transformers
    Found existing installation: transformers 5.1.0
    Uninstalling transformers-5.1.0:
      Successfully uninstalled transformers-5.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pandas<3.0,>=1.0, but you have pandas 3.0.0 which is incompatible.


In [3]:
# 1. Preferred installation method to avoid colab-new errors
!pip install unsloth
# 2. Force install dependencies to ensure compatibility
!pip install --no-deps xformers trl peft accelerate bitsandbytes
# 3. Required for metrics
!pip install -q -U evaluate rouge_score bert-score nltk

In [1]:
from unsloth import FastLanguageModel
import torch

model_id = "unsloth/gemma-2-9b-it-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    # CRITICAL: Use "unsloth" instead of True for 30% less VRAM
    use_gradient_checkpointing = "unsloth",
)
print("SUCCESS: Gemma-2-9B adapters attached!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Gemma2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.1.4 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


SUCCESS: Gemma-2-9B adapters attached!


In [3]:
import pandas as pd
from datasets import Dataset

# Load and split
df = pd.read_excel("/content/NewsSumm_Final_Cleaned.xlsx")
df = df.map(lambda x: str(x) if pd.notnull(x) else x)
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1, seed=42)

# Gemma-2 Instruction Format
gemma_prompt = """<start_of_turn>user
Summarize the following article:
{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn>"""

def format_prompts(examples):
    texts = [gemma_prompt.format(a, s) for a, s in zip(examples["article_text"], examples["human_summary"])]
    return { "text" : texts }

tokenized_dataset = dataset.map(format_prompts, batched = True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_dataset["train"],
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 1,     # Must stay 1 for 9B models on T4
        gradient_accumulation_steps = 16,    # High accumulation to keep effective batch size stable
        num_train_epochs = 3,
        learning_rate = 2e-4,
        fp16 = True,
        optim = "adamw_8bit",               # Mandatory for memory safety
        gradient_checkpointing = True,
        logging_steps = 10,
        output_dir = "gemma2_outputs",
        report_to = "none",
    ),
)
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/900 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 900 | Num Epochs = 3 | Total steps = 171
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 54,018,048 of 9,295,724,032 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.079700
20,1.909500
30,1.846100
40,1.851400
50,1.874200
60,1.773800
70,1.709400
80,1.660500
90,1.595400
100,1.664100


TrainOutput(global_step=171, training_loss=1.6795058166771604, metrics={'train_runtime': 7240.8881, 'train_samples_per_second': 0.373, 'train_steps_per_second': 0.024, 'total_flos': 7.402740522386227e+16, 'train_loss': 1.6795058166771604, 'epoch': 3.0})

In [8]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
# This forces the GPU to release the 14.72 GiB currently in use

In [5]:
import evaluate
from bert_score import score
rouge = evaluate.load("rouge")

def run_evaluation():
    FastLanguageModel.for_inference(model)
    preds, refs = [], list(dataset["test"]["human_summary"])

    print("Generating summaries...")
    for article in dataset["test"]["article_text"]:
        inputs = tokenizer([gemma_prompt.format(article, "")], return_tensors = "pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens = 128)
        decoded = tokenizer.batch_decode(outputs)[0]
        # Extract only the summary part
        summary = decoded.split("model\n")[-1].replace("<end_of_turn>", "").strip()
        preds.append(summary)

    # Calculate Results
    r = rouge.compute(predictions=preds, references=refs)
    P, R, F1 = score(preds, refs, lang="en", verbose=False)

    print(f"\nGemma-2-9B Results:\nROUGE-1: {r['rouge1']:.4f}\nROUGE-2: {r['rouge2']:.4f}\nROUGE-L: {r['rougeL']:.4f}\nBERTScore: {F1.mean().item():.4f}")

run_evaluation()

Generating summaries...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Gemma-2-9B Results:
ROUGE-1: 0.3904
ROUGE-2: 0.2116
ROUGE-L: 0.2916
BERTScore: 0.8790
